In [14]:
import MeCab
import os, json, random

dict_file = "markov.json"
dic = {}

# 辞書への登録
def regist_dic(wordlist) :
    global dic
    w1 =""
    w2 =""
    
    # 要素３未満何もしない
    if len(wordlist) < 3 : return
    for w in wordlist :
        word = w[0]
        if word == "" or word == "\r\n" or word == "\n" : continue
        # 辞書に単語設定
        if w1 and w2 :
            set_dic(dic, w1, w2, word)
        # 文末を表す語の場合、連鎖をクリア
        if word == "。" or word == "?" or word == "？"  :
            w1 =""
            w2 =""
            continue
            
        # 次の前後関係登録のためにスライド
        w1,w2 = w2, word
        
    # 辞書保存
    json.dump(dic, open(dict_file,"w", encoding = "utf-8"))
              

# 応答文の作成
def set_dic(dic, w1, w2, w3):
    # 新しい単語の場合は、新しい辞書オブジェクトを作成
    if w1 not in dic : dic[w1] = {}
    if w2 not in dic[w1] : dic[w1] [w2] = {}
    if w3 not in dic[w1][w2] : dic[w1] [w2] [w3]  = 0
   #  単語の出現数インクリメント
    dic[w1][w2][w3] += 1

# メイン処理
def make_response(word):
    res = []
    
    # 文章の意図を示すことが多い名詞、形容詞、動詞を始点とする
    w1 = word
    res.append(w1)
    w2 = word_choice(dic[w1])
    res.append(w2)
    while True:
        # w1,w2から予想される単語選択
        if w1 in dic and w2 in dic[w1] : w3 = word_choice(dic[w1][w2])
        else : w3 =""
        res.append(w2)
        # 文末を表す単語の場合、作文終了
        if w3 == "。" or w3 == "?" or w3 == "？"  : break
        # 次の前後関係登録のためにスライド
        w1,w2 = w2, w3
    #""を区切り文字にリストを連結して文字列を返す
    return "".join(res)

def word_choice(candidate):
    # keys()：キーと値のリストの取得
    keys = candidate.keys()
    # ランダムにリスト取得
    return random.choice(list(keys))

# メイン処理

# 辞書がすでにある場合最初に読み込み
if os.path.exists(dict_file):
    dic = json.load(open(dict_file, "r"))

while True:
    # 標準入力から入力を受けつけて、「さようなら」が入力されるまで続ける
    text = input("You -> ")
    if text == "" or text == "さようなら" :
        print("Bot -> 　さようなら")
        break

    # 文章整形　最後の文字が「。」とかでなければ「。」追加
    if text[-1] != "。" and text[-1]  != "?" and text[-1] != "？" : text+= "。"
    
    # 形態素解析
    tagger = MeCab.Tagger("-d /var/lib/mecab/dic/mecab-ipadic-neologd")
    tagger.parse("")
    node = tagger.parseToNode(text)
    
    # 形態素解析の結果から単語・品詞情報取得
    wordlist = []
    while node is not None :
        #　品詞の種類
        hinshi = node.feature.split(",")[0]
        # 文終わりを意味する:"BOS/EOS"
        if hinshi not in ["BOS/EOS"] :
            # []：リストの形　表層形,品詞
            wordlist.append([node.surface,hinshi])
        node = node.next

    # マルコフ連鎖の辞書に登録
    regist_dic(wordlist)

    # 応答文の作成
    for w in wordlist :
        # node.surface
        word = w[0]
        # node.hinshi
        hinshi = w[1]
        # 品詞が感動詞なら単語そのまま返す
        if hinshi in ["感動詞"] :
            print("Bot -> " + word)
            break
        # 品詞が「名詞」、「形容詞」、「動詞」で辞書に単語あるなら作文して返す
        elif (hinshi in ["名詞","形容詞","動詞"]) and (word in dic) :
            print("Bot -> " + make_response(word))
            break 

You -> mosimosi
You -> もしもし
Bot -> もしもし
You -> 銭湯行こう
Bot -> 銭湯行こ行こう
You -> 銭湯くる？
Bot -> 銭湯行こ行こう
You -> 
Bot -> 　さようなら
